<a href="https://colab.research.google.com/github/wdon021/From_Colab/blob/master/Word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
PATH = "/content/gdrive/My Drive/Fastai/"
import pandas as pd
h1b_raw = pd.read_csv(f'{PATH}H-1B_Disclosure_RAW_Data_FY18.csv', delimiter=',', engine='python')

In [0]:
import re

In [0]:
def removeAllPunctuations(g):
  g = g.str.replace(".", " ")
  g= g.str.replace(","," ")
  g= g.str.replace("'","")
  g= g.str.replace("-"," ")
  g= g.str.replace("/"," ")
  g= g.str.replace(":"," ")
  g= g.str.replace(";"," ")
  g= g.str.replace('"',"")
  g= g.str.replace("*","")
  g= g.str.replace("?"," ")
  g= g.str.replace("&"," and ")
  g= g.str.replace("+"," ")
  g= g.str.replace("["," ")
  g= g.str.replace("]"," ")
  g= g.str.replace("("," ")
  g= g.str.replace(")"," ")
  g= g.str.replace("<"," ")
  g= g.str.replace(">"," ")
  g= g.str.replace("="," ")
  g= g.str.replace(","," ")
  g= g.str.replace("{"," ")
  g= g.str.replace("}"," ")  
  # g= re.sub( '\s+', ' ', g ).strip()
  return g

In [0]:
# h1b_raw['JOB_TITLE'] = removeAllPunctuations(h1b_raw['JOB_TITLE'])

In [0]:
job = list(h1b_raw['JOB_TITLE'].str.lower().astype('category').cat.categories)

In [0]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [0]:
from nltk import word_tokenize

In [0]:
import nltk

In [11]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from gensim.models import word2vec, Phrases

In [0]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(job)]

In [0]:
max_epochs = 200
vec_size = 120
alpha = 0.025

In [21]:
model = Doc2Vec(size=vec_size,
                alpha=alpha,
                min_alpha=0.00025,
                min_count=1,
                dm=0) # 1 preseres the word order, 0 doesn't preserve any word order

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [0]:
model.build_vocab(tagged_data)

In [0]:
for epoch in range(max_epochs):
  print('iteration {0}'.format(epoch))
  model.train(tagged_data,
              total_examples=model.corpus_count,
              epochs=model.iter)
  # decrease the learning rate
  model.alpha -= 0.0002
  
model.save(f'{PATH}doc_100_no_clean.model')
print("model Saved")

iteration 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14


In [16]:
model = Doc2Vec.load(f'{PATH}doc_100.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
job_vec = list()

In [0]:
for jobvec in range(0, len(job)):
  job_vec.append(model.docvecs[jobvec])

In [0]:
doc2vector = pd.DataFrame(job_vec, index= job)

In [20]:
doc2vector.shape

(104574, 100)

In [21]:
!pip install hdbscan

     |████████████████████████████████| 4.9MB 5.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.23-cp36-cp36m-linux_x86_64.whl size=2332189 sha256=8db9420f9186573814354c9436616726643885d1d70c1e05d873aaf09cc6f497
  Stored in directory: /root/.cache/pip/wheels/1e/2a/69/0cdd5414624312666718f7526b0ab39cad65cea1c1b3892768
Successfully built hdbscan


In [0]:
import hdbscan

In [0]:
clusterer = hdbscan.HDBSCAN(metric='manhattan',  min_cluster_size=10)
clusterer.fit(doc2vector)
doc2vector['label'] = clusterer.labels_
doc2vector.to_csv(f'{PATH}hdbscan_100.csv')

In [24]:
doc2vector['label'].value_counts()

-1      94299
 191     3886
 100      168
 189      138
 108      112
        ...  
 139       10
 265       10
 59        10
 187       10
 84        10
Name: label, Length: 289, dtype: int64

In [0]:
import numpy as np
from sklearn.cluster import KMeans

In [26]:
kmean = KMeans(n_clusters=2000)
kmean.fit(doc2vector)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2000, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [0]:
doc2vector['label'] = kmean.labels_
doc2vector.to_csv(f'{PATH}kmean_100vec.csv')

In [0]:
docvec = model.docvecs[1]

In [0]:
print(docvec)

In [0]:
similar_doc = model.docvecs.most_similar('0')

In [0]:
model.vocabulary.

In [0]:
job['66796']

In [0]:
print(similar_doc)

In [0]:
trigram_sentences_project = []
bigram = Phrases(sentence_stream, min_count=1, delimiter=b' ')
trigram = Phrases(bigram[sentence_stream], min_count=1, delimiter=b' ')

In [0]:
for sent in sentence_stream:
  bigrams_ = bigram[sent]
  trigrams_ = trigram[bigram[sent]]
  trigram_sentences_project.append(trigrams_)

In [0]:
num_features = 10 # Word vector demensionality
min_word_count = 1 # minumum word count
num_workers = 4 # number of threads to run in parallel
context = 5 # context window size
downsampling = 1e-3 #downsample setting for frequent 

In [0]:
model = word2vec.Word2Vec(min_count=10, window=2, size=100, sample=6e-5, alpha=0.05, min_alpha=0.0007, negative=20)

In [0]:
model.build_vocab(trigram_sentences_project, progress_per=1000)

In [0]:
model.train(trigram_sentences_project, total_examples=model.corpus_count, epochs=30, report_delay=1)

In [0]:
model.wv.most_similar('scientist')

In [0]:
vocab = list(model.wv.vocab.keys())

In [0]:
print(vocab[:10])

In [0]:
job[0]

In [0]:
trigram_sentences_project

In [0]:
model.wv['&nbsp;application integration']

In [0]:
job[0]

In [0]:
for :
  print(i)